In [1]:
import os
import sys

CARTOLA_HOME = os.path.abspath('..')

print(CARTOLA_HOME)

if not os.path.isdir(CARTOLA_HOME):
    raise Exception('Cartola project is missing!')

sys.path.append(CARTOLA_HOME)

/home/cavani/Workspace/cartola-game


In [2]:
from bokeh.io import output_notebook, push_notebook, show
from bokeh.charts import HeatMap
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, HoverTool, SingleIntervalTicker, Span, Legend, LinearAxis, DataRange1d
from bokeh.plotting import figure

output_notebook()

Loading BokehJS ...

In [3]:
import gym
from cartola_game.gym import gym_init
from cartola_game.strategy import random_team, milp_team

gym_init()

env = gym.make('Cartola-v0')

# Load Season data
env.configure('2015')

[2017-05-24 20:45:07,609] Making new env: Cartola-v0


In [4]:
from collections import namedtuple

RunPlot = namedtuple('RunPlot', ['handle', 'money_source', 'score_source'])

# score_team - soma de pontos (score) do time
# money_networth - patrimônio, valor do time mais dinheiro em caixa
# money_team - valor do time
# money_free - dinheiro em caixa

def create_money_plot():
    data = ColumnDataSource(
        data=dict(
            round_number=[],
            money_networth=[],
            money_team=[],
            money_free=[]))
    hover = HoverTool(
        tooltips=[
            ('Round', '@round_number'),
            ('Networth', '@money_networth{1.11}'),
            ('Team Price', '@money_team{1.11}'),
            ('Free Money', '@money_free{1.11}'),
        ]
    )
    
    g = figure(width=950, height=300, tools=[hover], logo=None, toolbar_location=None,
               title='Money', x_axis_label='round', y_axis_label='money',
               x_range=DataRange1d(range_padding=0))
    g.xaxis[0].ticker = SingleIntervalTicker(interval=1)
    n = g.line('round_number', 'money_networth', color='blue', source=data)
    t = g.line('round_number', 'money_team', color='red', source=data)
    f = g.line('round_number', 'money_free', color='green', source=data)
    
    legend = Legend(items=[
        ('Networth', [n]),
        ('Team Price', [t]),
        ('Free Money', [f]),
    ], location=(0, -30))
    g.add_layout(legend, 'right')

    return g, data

def create_score_plot():
    data = ColumnDataSource(
        data=dict(
            round_number=[],
            score_team=[]))
    
    hover = HoverTool(
        tooltips=[
            ('Round', '@round_number'),
            ('Team Score', '@score_team{1.11}'),
        ]
    )
    g = figure(width=950, height=300, tools=[hover], logo=None, toolbar_location=None,
               title='Score', x_axis_label='round', y_axis_label='score',
               x_range=DataRange1d(range_padding=0))
    g.xaxis[0].ticker = SingleIntervalTicker(interval=1)

    #s = g.line('round_number', 'score_team', color='orange', source=data, y_range_name='Score')
    s = g.vbar('round_number', top='score_team', width=0.8, color='orange', source=data)

    hline = Span(location=0, dimension='width', line_color='black', line_width=1)
    g.add_layout(hline)
    
    legend = Legend(items=[
        ('Team Score', [s]),
    ], location=(0, -30))
    g.add_layout(legend, 'right')

    return g, data

def plot_run():
    money_g, money_data = create_money_plot()
    score_g, score_data = create_score_plot()
    h = show(column(money_g, score_g), notebook_handle=True)

    return RunPlot(h, money_data, score_data)

def stream_money(plot, round_number, money_networth, money_team, money_free):
    data = dict(
        round_number=[round_number],
        money_networth=[money_networth],
        money_team=[money_team],
        money_free=[money_free],
    )
    plot.money_source.stream(data)

def stream_score(plot, round_number, score_team):
    data = dict(
        round_number=[round_number],
        score_team=[score_team],
    )
    plot.score_source.stream(data)

def plot_stream(plot, round_number, money_networth, money_team, money_free, score_team):
    stream_money(plot, round_number, money_networth, money_team, money_free)
    stream_score(plot, round_number, score_team)
    push_notebook(handle=plot.handle)

def plot_formations(teams):
    rounds, formations = zip(*[(i+1, t.formation.label) for i, t in enumerate(teams)])
    hmap = HeatMap(dict(rounds=rounds, formations=formations), x='rounds', y='formations',
                   width=900, height=200, tools=None, legend=False, toolbar_location=None)
    hmap.x_range = DataRange1d(range_padding=0)
    hmap.xaxis[0].ticker = SingleIntervalTicker(interval=1)
    show(hmap)

In [5]:
def gym_run(env, agent_func, verbose=True):
    plot = plot_run()

    rewards = []
    actions = []

    # Initialize the Environment
    observation = env.reset()

    done = False
    while not done:
        round_number = observation.round_number
        
        money_networth = observation.total_money
        money_team = observation.team_price
        money_free = observation.free_money
    
        # Select a Team
        action = agent_func(observation)
        
        # Run one Round
        observation, reward, done, _ = env.step(action)

        rewards.append(reward)
        actions.append(action)

        # Update chart
        score_team = reward
        plot_stream(plot, round_number, money_networth, money_team, money_free, score_team)
    
    plot_formations(actions)
    
    print('...\n')
    print('Total reward: {:.2f}'.format(sum(rewards)))

def random_agent(observation):
    # Random team
    players = observation.data
    budget = observation.total_money
    return random_team(players, budget)

def milp_agent(observation):
    players = observation.data
    budget = observation.total_money
    if observation.round_number == 1:
        # Random first team
        return random_team(players, budget)
    else:
        # Best team from last round!
        candidates, scores, prices = \
            zip(*[(p.player, p.score, p.price)
                  for p in players.players_data.values()
                  if p.score > 0.0])
        return milp_team(candidates, scores, prices, budget, verbose=False)

def random_run(env):
    gym_run(env, random_agent)

def milp_run(env):
    gym_run(env, milp_agent)

In [6]:
random_run(env)

...

Total reward: 330.06


In [7]:
milp_run(env)

...

Total reward: 60.88
